<a href="https://colab.research.google.com/github/johanhoffman/methods-in-computational-science/blob/main/MICS_Eigenvalues_szaboda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Dániel Szabó**

# **Abstract**

This laboratory is mostly about solving a system of linear equations given in the form $Ax=b$ with the help of matrix factorization. So the goal is to write the martix $A$ as the product of multiple matrices, which makes it easier to calculate the solution. The mandatory tasks are the implementation of the sparse matrix-vector product, the QR factorization of a given matrix and the direct solving of system $Ax=b$. From the extra tasks, I chose to implement the QR eigenvalue algorithm.

#**About the code**

This report is written by Dániel Szabó, based on Johan Hoffman's template.

In [12]:
# Copyright (C) 2020,2021 Johan Hoffman (jhoffman@kth.se)
# Copyright (C) 2021 Dániel Szabó (dszabo@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This file is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# **Set up environment**

In [13]:
# Load neccessary modules.
import numpy as np
import random

random.seed(0)

# **Introduction**

Some details about the solved problems are presented here.

1. Sparse matrix-vector product: The input is a sparse, real, quadratic matrix $A\in\mathbb{R}^{n×n}$ and a real vector $x\in\mathbb{R}^n$. The task is to calculate their product $Ax$. The matrix $A$ is given in the CRS (Compressed Data Storage) format, that consists of three arrays: the first stores the nonzero values of $A$, the second their respective column indices, and the third contains the indices of the first nonzero values of each row, and one more element, that is the number of nonzero values of $A$ (plus one, if the indexing is one-based).

2. QR factorization: The input is a real, quadratic matrix $A\in\mathbb{R}^{n×n}$. The task is to calculate two matrices $Q\in\mathbb{R}^{n×n}$ and $R\in\mathbb{R}^{n×n}$, where $Q$ is orthogonal, $R$ is upper triangular and $A=QR$.

3. Direct solver of $Ax=b$: The input is a real, quadratic matrix $A\in\mathbb{R}^{n×n}$ and a real vector $b\in\mathbb{R}^n$. The task is to calculate $x\in\mathbb{R}^n$ that satisfies $Ax=b$.

4. QR eigenvalue algorithm: The input is a real, symmetric matrix $A\in\mathbb{R}^{n×n}$. The task is to calculate a vector $\lambda\in\mathbb{R}^n$ that contains the (real) eigenvalues of $A$, and a matrix $V\in\mathbb{R}^{n×n}$ that contains the (real) eigenvectors of $A$ as its columns.

# **Method**

Input validation: In most of the tasks we check if the input matrix is actually a matrix, i.e. if it has the same number of elements in each row, and it is done by the matrix_check function. Furthermore, in each task we also check if the input satisfies some other requirements, e.g. if the input matrix is a square matrix, if it is symmetric, if it is compatible with another input.

The first task is different from the input validation point of view, because here we have the matrix in the CRS format. It means that the input vectors should satisfy the constraints written in section 5.8 of the lecture notes -- with a bit of modification as now we have zero-based indexing. So, the number of elements in the val and the col_idx arrays should be the same; the last element of the row_ptr array should be the number of nonzero elements; the col_idx array should contain column indices; and the row_ptr array (except for the last element) should contain indices of the val array. This validation is done by the check_crs function.

In [14]:
def matrix_check(M):
    for i in range(len(M)):
        if(len(M[i])!=len(M[0])):
            raise Exception("The number of elements in each row of a matrix should be the same.")
    return
  
def check_crs(val, col_idx, row_ptr):
    n = len(row_ptr)-1
    nnz = len(val)
    if nnz != len(col_idx):
        raise Exception("CRS format not valid: the lengths of the first and second array should be the same.")
    if row_ptr[n] != nnz:
        raise Exception("CRS format not valid: the length of the first array should be the same as the last element of the third array.")
    for col in col_idx:
        if col>=n or col<0:
            raise Exception("CRS format not valid: the elements of the second array should be between 0 and the length of the trird array minus two.")
    for ptr in row_ptr[:n]:
        if ptr>=nnz or ptr<0:
            raise Exception("CRS format not valid: the elements of the third array should be between 0 and the length of the first array minus one.")

1. Sparse matrix-vector product: I created a class for storing matrices in the CRS format. The constructor checks if the input arrays are a valid CRS format, stores the three arrays and calculates the size of the martix as the length of the third array minus one.

 The method presented here follows Algorithm 5.9 of the lecture notes. The sparse matrix input of the method is given as an instance of the Crs_matrix class. First, the result vector is filled with zeros. The outer for loop iterates over the rows of the matrix, and the inner one iterates over the nonzero elements of the current row: these are stored in the first array of the CRS-matrix, between the two indices corresponding to the current row and the next one, and these indices are stored in the third array. These values are multiplied by the respective elements of the input vector, and the sum of these products is the respective element of the result.

In [15]:
class Crs_matrix:
    def __init__(self, values, column_indices, row_pointer):
        check_crs(values, column_indices, row_pointer)
        self.val = values
        self.col_idx = column_indices
        self.row_ptr = row_pointer
        self.n = len(row_pointer)-1

def sparse_matrix_vector_product(A, x):
    b = np.zeros(A.n)
    for i in range(A.n):
        for j in range(A.row_ptr[i], A.row_ptr[i+1]):
            b[i] += A.val[j]*x[A.col_idx[j]]
    return b

2. QR factorization: The method presented here validates the input and then follows Algorithm 5.3 of the lecture notes. First, $Q$ and $R$ are filled with zeros, and they have the same size as the input matrix. A for loop iterates over the columns of $A$. From each column, we subtract its projection of it onto the already calculated vectors of the orthonormal basis formed by the columns of $Q$. The lengths of these projections and the norm of the resulting vector $v$ are stored in $R$. The next column of $Q$ is the normalized version of $v$.

In [16]:
def QR_factorization(A):
    matrix_check(A)
    n = len(A)
    if n != len(A[0]):
        raise Exception("The input matrix should be a square matrix.")
    
    R = np.zeros((n,n))
    Q = np.zeros((n,n))
    for j in range(n):
        v = A[:,j]
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], v)
            v = np.subtract(v, R[i,j]*Q[:,i])
        R[j,j] = np.linalg.norm(v)
        Q[:,j] = np.divide(v, R[j,j])
    return Q, R

3. Direct solver of $Ax=b$: First, the validity of the input is checked. Then, using the method of the previous task, we calculate the QR facorization of the input matrix to get $QRx=b$. By multiplying both sides by $Q^{-1}=Q^T$ from the left, we get $Rx=Q^Tb$. The right hand side can be calculated easily (a transposition and a matrix-vector product), so we get a similar linear equation system as the original one ($Rx=b_2$), but now the matrix is upper triangular, so we can use backward substitution, which is Algorithm 5.2 in the lecture notes.

 The last element of the result vector is just the last element of $b_2$ divided by the last element of $R$. The calculation of the other elements of $x$ is also quite straightforward, if we know all the elements below the current one: for $x_i$, we subtract the inner product of the $i$'th row of $R$ and the already known part of $x$ (the not known elements are handled as zeros) from the $i$'th element of $b_2$, and divide the result by $R_{ii}$.

In [17]:
def direct_solver(A, b):
    matrix_check(A)
    n = len(A)
    if n != len(A[0]):
        raise Exception("The input matrix should be a square matrix.")
    if n != len(b):
        raise Exception("The number of elements in the input vector should be the same as the size of the matrix.")

    Q, R = QR_factorization(A)
    Qinv = np.transpose(Q)
    b2=np.dot(Qinv, b)
    x = np.zeros(n)
    x[n-1]=b2[n-1]/R[n-1,n-1]
    for i in range(n-2, -1, -1):
        sum = 0
        for j in range(n-i-1):
            sum += R[i, i+1+j]*x[i+1+j]
        x[i]=(b2[i]-sum)/R[i,i]
    return x

4. QR eigenvalue algorithm: The method presented here validates the input, then it follows Algorithm 6.1 of the lecture notes. The input is the matrix and a positive real number $\varepsilon$ that determines the required precision of the results. In the beginning, $U$ is the identity matrix of the same size as the input matrix. In a while loop, we calculate the QR factorization of $A$ using the method of task 2. Matrix $A_2=RQ$ is calculated, and we look at how much it is different from $A$: if the Frobenius norm of $A-A_2$ is small enough (not greater than $\varepsilon$), we will not have a next iteration. $A$ is substituted by $A_2$, and $U$ is updated as $UQ$. When the iterations are over, we have the (approximate) results: the eigenvalues are in the diagonal of $A$, and the eigenvectors are the columns of $U$.

In [18]:
def QR_eigenvalue(A, eps):
    matrix_check(A)
    n = len(A)
    if n != len(A[0]):
        raise Exception("The input matrix should be a square matrix.")
    if eps<=0:
        raise Exception("Epsilon should be positive.")
    for i in range(n):
        for j in range(i):
            if A[i,j]!=A[j,i]:
                raise Exception("The input matrix should be symmetric.")
                
    U = np.eye(n)
    change = np.inf
    while(change > eps):
        Q, R = QR_factorization(A)
        A2 = np.dot(R,Q)
        change = np.linalg.norm(np.subtract(A, A2))
        A = A2
        U = np.dot(U,Q)
    eig_val = A.diagonal()
    return eig_val, U

# **Results**

The verification of the methods is done by computing the results for some random test data using the algorithms of the Method section and then calculating the error metrics that are required in the lab instructions using some numpy methods. Then we check if these values are close to 0. Some tests with invalid input are also provided.

1. Sparse matrix-vector product: The test matrix is the one from Example 5.5 of the lecture notes (it would have been difficult to generate a random sparse matrix in both the CRS and the usual formats). We calculate its product with a random vector using our sparse matrix-vector product method and also with the numpy dot function. It is checked if the two result vectors are close to each other. Finally, there are some tests checking is the input validation is working properly.

In [19]:
A1 = Crs_matrix([3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3], [0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 5], [0, 3, 5, 6, 8, 9, 11])
x = [random.gauss(0, 1) for _ in range(6)]
b1 = sparse_matrix_vector_product(A1, x)
print(b1)
# A is the same matrix as A1, but in the usual form
A=[[3, 2, 0, 2, 0, 0],[0, 2, 1, 0, 0, 0],[0, 0, 1, 0, 0, 0],[0, 0, 3, 2, 0, 0],[0, 0, 0, 0, 1, 0],[0, 0, 0, 0, 2, 3]]
b2 = np.dot(A,x)
print(b2)
np.testing.assert_almost_equal(b1, b2)

# Invalid input test: there is a value 7 in the second array
np.testing.assert_raises(Exception, Crs_matrix, [3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3], [0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 7], [0, 3, 5, 6, 8, 9, 11])
# Invalid input test: the second array has more elements than the first one
np.testing.assert_raises(Exception, Crs_matrix, [3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3], [0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 5, 0], [0, 3, 5, 6, 8, 9, 11])
# Invalid input test: the last element of the third array is not 11
np.testing.assert_raises(Exception, Crs_matrix, [3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3], [0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 5], [0, 3, 5, 6, 8, 9, 12])
# Invalid input test: there is a value 11 in the third array before the last element
np.testing.assert_raises(Exception, Crs_matrix, [3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3], [0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 5], [0, 3, 5, 6, 8, 11, 11])

[ 0.77299714 -3.47287066 -0.67971445 -1.29813621 -1.01634889 -2.24905786]
[ 0.77299714 -3.47287066 -0.67971445 -1.29813621 -1.01634889 -2.24905786]


2. QR factorization: After calculating the QR factorizaton of a random matrix with our method, we check if the resulting $R$ matrix is upper triangular and if norms $||Q^TQ-I||_F$ and $||QR-A||_F$ are close to zero. Some tests check if we get an exception for giving invalid input to the function.

In [20]:
n = random.randint(2,5)
A = np.array([[random.gauss(0, 1) for _ in range(n)] for __ in range(n)])
Q, R = QR_factorization(A)
print("A =",A)
print("Q =",Q)
print("R =",R)

for i in range(n):
        for j in range(i):
            np.testing.assert_almost_equal(R[i,j], 0)
norm1 = np.linalg.norm(np.subtract(np.dot(np.transpose(Q), Q), np.eye(n)))
norm2 = np.linalg.norm(np.subtract(np.dot(Q, R), A))
print("errors:", norm1, norm2)
np.testing.assert_almost_equal(norm1, 0)
np.testing.assert_almost_equal(norm2, 0)

# Invalid input test: non-square matrix
np.testing.assert_raises(Exception, QR_factorization, np.array([[1,2],[3,4],[5,6]]))
# Invalid input test: not valid matrix
np.testing.assert_raises(Exception, QR_factorization, np.array([[1,2],[3,4,0],[5,6]]))

A = [[ 0.9223306  -0.18919311  0.54561157 -0.44186164]
 [ 0.35167029  0.42116823  1.48357879  1.00593461]
 [ 1.22933224 -0.09862225 -0.37724849 -1.30376305]
 [ 0.27504975  0.3637615   1.74396019  0.87130391]]
Q = [[ 0.57631303 -0.28905001  0.72088699  0.25423478]
 [ 0.21973918  0.71850234 -0.11663922  0.64951087]
 [ 0.76814126 -0.12731881 -0.58604646 -0.22427319]
 [ 0.17186327  0.61967282  0.35109368 -0.68058934]]
R = [[ 1.60039866 -0.02972587  0.65038602 -0.88533643]
 [ 0.          0.59526621  2.03696136  1.55640338]
 [ 0.          0.          1.05365936  0.63411129]
 [ 0.          0.          0.          0.24042782]]
errors: 1.2116783524244355e-15 2.665835984225005e-16


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


3. Direct solver of $Ax=b$: For a random matrix $A$ and a random vector $b$, the solution is calculated with our method, then we check if norms $||Ax-b||$ and $||x-y||$ are close to zero, where $y=A^{-1}b$. Invalid input handling is also tested.

In [21]:
n = random.randint(2,5)
A = np.array([[random.gauss(0, 1) for _ in range(n)] for __ in range(n)])
b = [random.gauss(0, 1) for _ in range(n)]
x = direct_solver(A,b)
print("A =", A)
print("b =", b)
print("x =", x)

norm1 = np.linalg.norm(np.subtract(np.dot(A,x), b))
norm2 = np.linalg.norm(np.subtract(x, np.dot(np.linalg.inv(A), b)))
print("errors:", norm1, norm2)
np.testing.assert_almost_equal(norm1, 0)
np.testing.assert_almost_equal(norm2, 0)

# Invalid input test: not compatible inputs
np.testing.assert_raises(Exception, direct_solver, np.array([[1,2],[3,4]]), np.array([1,2,3]))
# Invalid input test: non-square matrix
np.testing.assert_raises(Exception, direct_solver, np.array([[1,2],[3,4],[5,6]]), np.array([1,2]))

A = [[-0.45369873  0.08023304 -1.2581033   0.55222001]
 [ 2.22757729 -1.35524149 -1.98153308  0.28824375]
 [-0.11912331  1.80432993 -0.16036218 -0.05065971]
 [-0.19087389 -0.99060624  0.67302998 -1.32408246]]
b = [1.166490138238534, 0.008376179502044914, 0.5036300563785048, -0.5527646971207058]
x = [-0.53332405  0.17898729 -0.72755226 -0.00937108]
errors: 7.378837525927943e-16 4.0395225436151075e-16


4. QR eigenvalue algorithm: First, we calculate the eigenvalues and eigenvectors of a random symmetric matrix using our method. Then it is checked for each eigenvalue-eigenvector pair $(\lambda_i,v_i)$ if $\det(A-\lambda_iI)$ and $||Av_i-\lambda_iv_i||$ are close to zero. Here, the precision parameter $\varepsilon$ is used to determine the number of digits that have to be zero (instead of the default 7 digits), because the precision of the algorithm depends on $\varepsilon$. We also test the handling of invalid input.

In [22]:
n = random.randint(2,5)
A = np.array([[random.gauss(0, 1) for _ in range(n)] for __ in range(n)])
for i in range(n):
    for j in range(i):
        A[i,j]=A[j,i]
epsilon=10**-5
val, vec = QR_eigenvalue(A, epsilon)
print("A =", A)
print("eigenvalues:", val)
print("eigenvectors:", vec)

for lambdai, vi in zip(val, np.transpose(vec)):
    det=np.linalg.det(np.subtract(A, lambdai*np.eye(n)))
    norm=np.linalg.norm(np.subtract(np.dot(A, vi), lambdai*vi))
    print("errors:", det, norm)
    np.testing.assert_almost_equal(det, 0, decimal=np.log10(1/epsilon)-1)
    np.testing.assert_almost_equal(norm, 0, decimal=np.log10(1/epsilon)-1)

# Invalid input test: not symmetric matrix
np.testing.assert_raises(Exception, QR_eigenvalue, np.array([[1,2],[3,4]]), 10**-3)
# Invalid input test: non-positive epsilon
np.testing.assert_raises(Exception, QR_eigenvalue, np.array([[1,2],[2,4]]), 0)

A = [[-0.10358506 -0.34551492  0.61394797 -0.3526533 ]
 [-0.34551492 -0.52363039 -0.42151456 -0.11670926]
 [ 0.61394797 -0.42151456  1.15917033 -0.01477398]
 [-0.3526533  -0.11670926 -0.01477398  0.63531794]]
eigenvalues: [ 1.55726963 -0.77481938  0.72020249 -0.33537992]
eigenvectors: [[ 0.40147631 -0.51517047 -0.20271505 -0.72960453]
 [-0.23601139 -0.83364822 -0.1054549   0.48806612]
 [ 0.8741592  -0.01967572  0.21422533  0.43539185]
 [-0.13769924 -0.19809823  0.94968067 -0.19975643]]
errors: 1.894769189414771e-15 8.746309929968238e-16
errors: -8.014211955015597e-09 8.843176618869393e-05
errors: 6.909856871685709e-09 8.843176618855327e-05
errors: -7.10280573089439e-18 7.381365600396372e-16


# **Discussion**

The results were exactly as expected. All the methods implemented for solving the tasks, actually succeeded in solving them, as it is confirmed by the test results. Furthermore, the inputs are validated properly.